<a href="https://colab.research.google.com/github/waelrash1/predictive_analytics_DT302/blob/main/hateSpeech/HateSpeechFineTune_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
waalbannyantudre_hate_speech_detection_curated_dataset_path = kagglehub.dataset_download('waalbannyantudre/hate-speech-detection-curated-dataset')

print('Data source import complete.')


# Transfer learning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import torch

# Load your dataset
dataset_path = '/kaggle/input/hate-speech-detection-curated-dataset/HateSpeechDatasetBalanced.csv'
data = pd.read_csv(dataset_path)
print("Original dataset shape:", data.shape)

# Perform stratified sampling for p% of the data, to expalin the idea
# in your case use the datset as whole but you need to manage the computation
sampled_data, _ = train_test_split(
    data,
    test_size=0.98,                 # Leave out 80%, keep 20%
    stratify=data['Label'],        # Maintain class distribution
    random_state=42                # For reproducibility
)

# Output the sampled dataset details
print("Sampled dataset shape:", sampled_data.shape)
print("Class distribution in sampled data:")
print(sampled_data['Label'].value_counts(normalize=True))  # Verify class balance
data=sampled_data

Original dataset shape: (726119, 2)
Sampled dataset shape: (14522, 2)
Class distribution in sampled data:
Label
1    0.501997
0    0.498003
Name: proportion, dtype: float64


In [ ]:
# Train-test split
train_x, test_texts, train_y, test_labels = train_test_split(
    data['Content'], data['Label'], test_size=0.2, random_state=42
)

# Train-Val split
train_texts, val_texts, train_labels, val_labels = train_test_split(
   train_x, train_y,  test_size=0.2, random_state=42
)

print("Train dataset shape:", train_texts.shape)
print("Val dataset shape:", val_texts.shape)
print("Test dataset shape:", test_texts.shape)

Train dataset shape: (9293,)
Val dataset shape: (2324,)
Test dataset shape: (2905,)


# Fine-Tune Model
The transformers library allows you to fine-tune a model on a specific task using classes like Trainer or custom PyTorch code.


> **The GroNLP/hateBER**T model has approximately **110 million** parameters. This is based on its architecture, which is derived from the BERT base uncased model, a standard transformer architecture with **12 layers, 12 attention heads** , and a hidden size of **768**.

## Approach 1: Using the Trainer API
* This simplifies the process:

1. Convert your dataset into Dataset objects.
1. Define a model head for you1. r task (e.g., classification).

In [ ]:

# Convert data to Hugging Face Dataset
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:

# Tokenize
def preprocess(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)
train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

# Load model for classification
model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2).to(device)

Map:   0%|          | 0/9293 [00:00<?, ? examples/s]

Map:   0%|          | 0/2324 [00:00<?, ? examples/s]

Map:   0%|          | 0/2905 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Define evaluation metric
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR API key must be 40 characters long, yours was 4
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.406000,0.346931,0.847246
2,0.378300,0.521840,0.827022
3,0.175800,0.871206,0.825731
4,0.002700,0.961082,0.829174


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=2324, training_loss=0.1925801417147965, metrics={'train_runtime': 2398.9164, 'train_samples_per_second': 15.495, 'train_steps_per_second': 0.969, 'total_flos': 6927839476491120.0, 'train_loss': 0.1925801417147965, 'epoch': 4.0})

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Generate predictions on the test set
predictions = trainer.predict(test_dataset)

# Extract logits and true labels
logits = predictions.predictions
true_labels = predictions.label_ids

# Convert logits to predicted class labels
predicted_labels = np.argmax(logits, axis=-1)

# Generate classification report
report = classification_report(true_labels, predicted_labels, target_names=["Class 0", "Class 1"])
print("Classification Report:")
print(report)

# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_hatebert")
tokenizer.save_pretrained("./fine_tuned_hatebert")


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Classification Report:
              precision    recall  f1-score   support

     Class 0       0.86      0.83      0.85      1483
     Class 1       0.83      0.86      0.85      1422

    accuracy                           0.85      2905
   macro avg       0.85      0.85      0.85      2905
weighted avg       0.85      0.85      0.85      2905



('./fine_tuned_hatebert/tokenizer_config.json',
 './fine_tuned_hatebert/special_tokens_map.json',
 './fine_tuned_hatebert/vocab.txt',
 './fine_tuned_hatebert/added_tokens.json',
 './fine_tuned_hatebert/tokenizer.json')

In [ ]:
import torch

# Clear GPU memory
torch.cuda.empty_cache()

## Approach 2: Custom Fine-Tuning with PyTorch
If you prefer more control:

* Add a classification head.
* D efine a custom training loop.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
# Ensure tokenizer is defined before the preprocess function
from transformers import AutoTokenizer
# Load hateBERT tokenizer and model
base_model = "GroNLP/hateBERT"
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModel.from_pretrained(base_model)



In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim

# Define Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]


# Prepare data
train_dataset = TextDataset(train_texts.tolist(), train_labels.tolist())
val_dataset = TextDataset(val_texts.tolist(), val_labels.tolist())
test_dataset=TextDataset(test_texts.tolist(), test_labels.tolist())

In [ ]:
#Loading Data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader= DataLoader(test_dataset, batch_size=16)

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch.optim as optim

# Initialize the base model
base_model_name = "GroNLP/hateBERT"
base_model = AutoModel.from_pretrained(base_model_name)

# Define the classification head
class HateBERTClassifier(nn.Module):
    def __init__(self, base_model, num_labels):
        super(HateBERTClassifier, self).__init__()
        self.base = base_model
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, **kwargs):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state.mean(dim=1)
        return self.classifier(pooled_output)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Instantiate the model
num_labels = 2  # Binary classification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HateBERTClassifier(base_model, num_labels).to(device)


In [ ]:
# Training loop
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(4):
    model.train()
    for batch in train_loader:
        texts, labels = batch
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    model.eval()
    val_accuracy = 0
    with torch.no_grad():
        for batch in val_loader:
            texts, labels = batch
            inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs)
            predictions = torch.argmax(outputs, dim=-1)
            val_accuracy += (predictions == labels).sum().item()

    val_accuracy /= len(val_dataset)
    print(f"Epoch {epoch + 1}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1, Validation Accuracy: 0.7810
Epoch 2, Validation Accuracy: 0.8511
Epoch 3, Validation Accuracy: 0.7904
Epoch 4, Validation Accuracy: 0.8330


In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
model.eval()
all_test_predictions = []
all_test_labels = []
with torch.no_grad():
    for batch in test_loader:
        texts, labels = batch
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs)
        predictions = torch.argmax(outputs, dim=-1)

        # Collect predictions and labels for classification report
        all_test_predictions.extend(predictions.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# Calculate test metrics
test_accuracy = (np.array(all_test_predictions) == np.array(all_test_labels)).mean()
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate classification report
class_names = ['Neutral', 'Hate']
report = classification_report(all_test_labels, all_test_predictions, target_names=class_names)
print(f"Classification Report on Test Set:\n{report}")



Evaluating on test set...
Test Accuracy: 0.8482
Classification Report on Test Set:
              precision    recall  f1-score   support

     Neutral       0.85      0.85      0.85      1483
        Hate       0.85      0.84      0.84      1422

    accuracy                           0.85      2905
   macro avg       0.85      0.85      0.85      2905
weighted avg       0.85      0.85      0.85      2905



## Approach 3: Freez base-model parameters
- freez the base model parameters
- add couple of lstm and Dense layers
- optimise the parameters of the newely added layers

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
import torch.optim as optim

# Initialize the base model
base_model_name = "GroNLP/hateBERT"  # Replace with your model's name
base_model = AutoModel.from_pretrained(base_model_name)


# Define the classification model with two LSTM layers and three dense layers
class HateBERTClassifier(nn.Module):
    def __init__(self, base_model, num_labels, lstm_hidden_size=128, dense_hidden_size=64, dropout_prob=0.3):
        super(HateBERTClassifier, self).__init__()
        self.base = base_model

        # Freeze pretrained parameters
        for param in self.base.parameters():
            param.requires_grad = False

        # First LSTM layer
        self.lstm1 = nn.LSTM(
            input_size=self.base.config.hidden_size,
            hidden_size=lstm_hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )

        # Second LSTM layer
        self.lstm2 = nn.LSTM(
            input_size=lstm_hidden_size * 2,
            hidden_size=lstm_hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )

        # Fully connected dense layers
        self.dense1 = nn.Linear(lstm_hidden_size * 2, dense_hidden_size)
        self.dense2 = nn.Linear(dense_hidden_size, dense_hidden_size)
        self.dense3 = nn.Linear(dense_hidden_size, dense_hidden_size)

        # Dropout
        self.dropout = nn.Dropout(dropout_prob)

        # Final output layer
        self.output_layer = nn.Linear(dense_hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, **kwargs):
        # Get base model outputs
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)

        # Pass through the first LSTM layer
        lstm1_output, _ = self.lstm1(sequence_output)  # Shape: (batch_size, seq_len, 2*lstm_hidden_size)

        # Pass through the second LSTM layer
        lstm2_output, _ = self.lstm2(lstm1_output)  # Shape: (batch_size, seq_len, 2*lstm_hidden_size)

        # Pooling: Use the mean of the second LSTM outputs
        pooled_output = lstm2_output.mean(dim=1)  # Shape: (batch_size, 2*lstm_hidden_size)

        # Pass through dense layers
        x = self.dropout(pooled_output)
        x = torch.relu(self.dense1(x))
        x = self.dropout(torch.relu(self.dense2(x)))
        x = self.dropout(torch.relu(self.dense3(x)))

        # Final output layer
        return self.output_layer(x)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Instantiate the model
num_labels = 2  # Binary classification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HateBERTClassifier(base_model, num_labels).to(device)


In [ ]:
print(model)

HateBERTClassifier(
  (base): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# Optimizer and loss
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in tqdm(range(4)):
    model.train()
    for batch in train_loader:
        texts, labels = batch
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items() if key != "token_type_ids"}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on validation set
    model.eval()
    val_accuracy = 0
    with torch.no_grad():
        for batch in val_loader:
            texts, labels = batch
            inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items() if key != "token_type_ids"}
            labels = labels.to(device)

            outputs = model(**inputs)
            predictions = torch.argmax(outputs, dim=-1)
            val_accuracy += (predictions == labels).sum().item()

    val_accuracy /= len(val_dataset)
    print(f"Epoch {epoch + 1}, Validation Accuracy: {val_accuracy:.4f}")


 25%|██▌       | 1/4 [02:26<07:20, 146.77s/it]

Epoch 1, Validation Accuracy: 0.7620


 50%|█████     | 2/4 [04:53<04:53, 146.64s/it]

Epoch 2, Validation Accuracy: 0.7754


 75%|███████▌  | 3/4 [07:20<02:26, 146.93s/it]

Epoch 3, Validation Accuracy: 0.7775


100%|██████████| 4/4 [09:46<00:00, 146.72s/it]

Epoch 4, Validation Accuracy: 0.7565


In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
model.eval()
all_test_predictions = []
all_test_labels = []
with torch.no_grad():
    for batch in test_loader:
        texts, labels = batch
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs)
        predictions = torch.argmax(outputs, dim=-1)

        # Collect predictions and labels for classification report
        all_test_predictions.extend(predictions.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# Calculate test metrics
test_accuracy = (np.array(all_test_predictions) == np.array(all_test_labels)).mean()
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate classification report
class_names = ['Neutral', 'Hate']
report = classification_report(all_test_labels, all_test_predictions, target_names=class_names)
print(f"Classification Report on Test Set:\n{report}")

Evaluating on test set...
Test Accuracy: 0.7972
Classification Report on Test Set:
              precision    recall  f1-score   support

     Neutral       0.80      0.80      0.80      1483
        Hate       0.79      0.80      0.79      1422

    accuracy                           0.80      2905
   macro avg       0.80      0.80      0.80      2905
weighted avg       0.80      0.80      0.80      2905

